# ML Instant House Valuation: Part 2 - Data Sourcing
> In this section, I will be collecting data from numerous sourcing and stitching them together in one dataset to use later for machine learning modelling. I will use open data from the HM Land Registry, and various other public sources.

- toc:true
- branch: master
- badges: true
- author: Uzair Patel
- categories: [house-prices]

In [ ]:
# collapse
import os, shutil, zipfile
from requests import Session
from tqdm import tqdm

import numpy as np
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

In [ ]:
# hide
RAW_PATH = "data/raw"
PARTS_PATH = "data/parts"
os.makedirs(RAW_PATH, exist_ok=True)
os.makedirs(PARTS_PATH, exist_ok=True)

PPD_PATH = os.path.join(PARTS_PATH, "pricepaid")
EPC_PATH = os.path.join(PARTS_PATH, "epc")
MERGED_PATH = os.path.join(PARTS_PATH, "merged")
PCD_PATH = os.path.join(RAW_PATH, "postcode-lookup.csv")
HPI_PATH = os.path.join(RAW_PATH, "house-price-index.csv")


In [ ]:
# hide
# General information on dataframe (datatypes, memory usage, nulls, number of unique values)
def info(df):
    return pd.DataFrame(
        dict(dtype=df.dtypes,
             memory_usage=df.memory_usage(deep=True),
             null=df.isna().sum(),
             unique=df.nunique())).reindex([*df.columns, "Index"])


# Convert bytes integer to bytes string format
def to_bytes(size):
    for x in ['bytes', 'KB', 'MB', 'GB']:
        if size < 1000:
            return f"{size:.2f}{x}"
        size /= 1000.
    return size

##  Price paid data
[***HM Land Registry price paid data***](https://www.gov.uk/guidance/about-the-price-paid-data) tracks the property sales in England and Wales submitted to HM Land Registry for registration, based on the raw data released every month, with data from 1995 to now (January 2022).

Column headers:
* Price
* Date of Transfer
* Postcode
* Property Type: *D* (Detached), *S* (Semi-Detached), *T* (Terraced), *F* (Flats/Maisonettes), *O* (Other)
* Old/New: *Y* (New), *N* (Old)
* PAON - Primary Addressable Object Name (e.g house number)
* SAON - Secondary Addressable Object Name (e.g unit number)
* Street
* Locality
* Town/City
* District

### Sampling data
Initially, I will take a small chunk of the first 50 thousand rows to test and apply transformations before ultimately applying these transformations to the large dataset. Currently, I will largely be adjusting the dataset to minimise its memory usage by dropping columns and converting data types.

In [ ]:
ppdUrl = "http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-complete.csv"
ppdNames = [
    "UID", "PRICE", "DATE_OF_TRANSFER", "POSTCODE", "PROPERTY_TYPE", "OLD_NEW",
    "DURATION", "PAON", "SAON", "STREET", "LOCALITY", "TOWN_CITY", "DISTRICT",
    "COUNTY", "PPD_CAT", "RECORD_STATUS"
]
ppdSample = dd.read_csv(ppdUrl, names=ppdNames, dtype=object).head(n=50_000)

In [ ]:
# hide_input
originalUsage = to_bytes(ppdSample.memory_usage(deep=True).sum())
print(originalUsage)
ppdSample.head()

50.77MB


,UID,PRICE,DATE_OF_TRANSFER,POSTCODE,PROPERTY_TYPE,OLD_NEW,DURATION,PAON,SAON,STREET,LOCALITY,TOWN_CITY,DISTRICT,COUNTY,PPD_CAT,RECORD_STATUS
0,{5BBE9CB3-6332-4EB0-9CD3-8737CEA4A65A},42000,1995-12-21 00:00,NE4 9DN,S,N,F,8,NaN,MATFEN PLACE,FENHAM,NEWCASTLE UPON TYNE,NEWCASTLE UPON TYNE,TYNE AND WEAR,A,A
1,{20E2441A-0F16-49AB-97D4-8737E62A5D93},95000,1995-03-03 00:00,RM16 4UR,S,N,F,30,NaN,HEATH ROAD,GRAYS,GRAYS,THURROCK,THURROCK,A,A
2,{D893EE64-4464-44B5-B01B-8E62403ED83C},74950,1995-10-03 00:00,CW10 9ES,D,Y,F,15,NaN,SHROPSHIRE CLOSE,MIDDLEWICH,MIDDLEWICH,CONGLETON,CHESHIRE,A,A
3,{F9F753A8-E56A-4ECC-9927-8E626A471A92},43500,1995-11-14 00:00,TS23 3LA,S,N,F,19,NaN,SLEDMERE CLOSE,BILLINGHAM,BILLINGHAM,STOCKTON-ON-TEES,STOCKTON-ON-TEES,A,A
4,{E166398A-A19E-470E-BB5A-83B4C254CF6D},63000,1995-09-08 00:00,CA25 5QH,S,N,F,8,NaN,CROSSINGS CLOSE,CLEATOR MOOR,CLEATOR MOOR,COPELAND,CUMBRIA,A,A


#### Dropping empty entries and unnecessary columns
The only important columns in the dataset are the price and dates, the location data is not particularly helpful and I will be substituting them for coordinates instead.

In [ ]:
ppdFilter = [
    "PRICE",
    "DATE_OF_TRANSFER",
    "POSTCODE",
    "OLD_NEW",
    "PAON",  #*
    "SAON",  #*
    "STREET"  #*
]
#* WILL LATER BE DROPPED
ppdSample = ppdSample[ppdFilter]

In [ ]:
# Dropping null entries (ignoring SAON)
print(ppdSample.isna().sum()[lambda x: x > 0])
l = ppdSample.shape[0]
ppdSample = (ppdSample
    .dropna(subset=list(filter(lambda x: x != "SAON", ppdFilter)))
    .reset_index(drop=True))


POSTCODE       36
PAON            5
SAON        46280
STREET        750
dtype: int64


#### Converting data types

In [ ]:
ppdSample["PRICE"] = ppdSample.PRICE.astype(
    "float")  # min max ~2bn | float32 - 4 bytes
ppdSample["DATE_OF_TRANSFER"] = dd.to_datetime(
    ppdSample.DATE_OF_TRANSFER)  # Converting from string to datetime
ppdSample["OLD_NEW"] = ppdSample.OLD_NEW.astype("category")
ppdDtype = ppdSample.dtypes.to_dict()
ppdDtype["DATE_OF_TRANSFER"] = np.dtype("O")

#### Address column
Merging `PAON`, `SAON`, `STREET` and `POSTCODE` into one `ADDRESS` column. I will apply the same formatting of "`SAON PAON STREET POSTCODE`" to the addresses in the EPC dataset in order to cross-reference and merge the datasets into one.

In [ ]:
def ppdFormatAddr(x):
    return [
        ' '.join(z.strip() for z in y if isinstance(z, str))
        for y in x[["SAON", "PAON", "STREET", "POSTCODE"]].values
    ]  # List comprehension for performance

In [ ]:
ppdSample["ADDRESS"] = ppdFormatAddr(ppdSample)
# Keeping POSTCODE column for geolocation purposes
ppdSample = ppdSample.drop(columns=["PAON", "SAON", "STREET"])
ppdSample.ADDRESS[:5]

0          8 MATFEN PLACE NE4 9DN
1          30 HEATH ROAD RM16 4UR
2    15 SHROPSHIRE CLOSE CW10 9ES
3      19 SLEDMERE CLOSE TS23 3LA
4      8 CROSSINGS CLOSE CA25 5QH
Name: ADDRESS, dtype: object

Processed sample set:

In [ ]:
# hide_input
_info = info(ppdSample)
print(originalUsage, "=>", to_bytes(_info["memory_usage"].sum()))
_info

50.77MB => 8.09MB


,dtype,memory_usage,null,unique
PRICE,float64,393736,0.0,2900.0
DATE_OF_TRANSFER,datetime64[ns],393736,0.0,356.0
POSTCODE,object,3171295,0.0,45548.0
OLD_NEW,category,49441,0.0,2.0
ADDRESS,object,4085965,0.0,49158.0
Index,NaN,128,NaN,NaN


### Entire dataset
I will process and export the dataset using Dask, which utilises parallelisation accross multiple threads to enhance performance.

#### Pipeline
1. Import the entire dataset with preset datatypes and already filtered columns.
2. Drop rows with empty entries (ignoring `SAON` column which is largely empty)
3. Format addresses in each partition with `ppdFormatAddr` function and set as index
4. Drop duplicate addresses, keeping last.
5. Drop remaining unneccessary columns.
8. Store dataframe

In [ ]:
ppdUrl = "http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-complete.csv"
ppdNames = [
    "UID", "PRICE", "DATE_OF_TRANSFER", "POSTCODE", "PROPERTY_TYPE", "OLD_NEW",
    "DURATION", "PAON", "SAON", "STREET", "LOCALITY", "TOWN_CITY", "DISTRICT",
    "COUNTY", "PPD_CAT", "RECORD_STATUS"
]
ppdFilter = [
    "PRICE", "DATE_OF_TRANSFER", "POSTCODE", "OLD_NEW", "PAON", "SAON",
    "STREET"
]
ppdDtype = {
    "PRICE": np.float64,
    "DATE_OF_TRANSFER": object,
    "POSTCODE": object,
    "OLD_NEW": pd.CategoricalDtype(),
    "PAON": object,
    "SAON": object,
    "STREET": object
}
def ppdFormatAddr(x):
    return [
        ' '.join(z.strip() for z in y if isinstance(z, str))
        for y in x[["SAON", "PAON", "STREET", "POSTCODE"]].values
    ]  # List comprehension for performance

In [ ]:
ppdPipeline = (dd
    .read_csv(ppdUrl, names=ppdNames, usecols=ppdFilter, parse_dates=["DATE_OF_TRANSFER"], dtype=ppdDtype)
    .dropna(subset=filter(lambda x: x != "SAON", ppdFilter))
    .map_partitions(lambda x: x.assign(ADDRESS=ppdFormatAddr))
    .drop_duplicates(subset="ADDRESS",keep="last")
    .map_partitions(lambda x: x.set_index("ADDRESS"))
    .drop(columns=["SAON", "PAON", "STREET"])
)

In [ ]:
if not os.path.exists(PPD_PATH) and not os.path.exists(DATASET_PATH):
    with ProgressBar():
        ppdPipeline.to_parquet(PPD_PATH, compression="snappy")
ppd = dd.read_parquet(PPD_PATH)

In [ ]:
# hide_input
print(to_bytes(ppd.memory_usage(deep=True).sum().compute()))
print(f"{ppd.shape[0].compute():,} rows")
ppd.head()

2.46GB
14,876,050 rows


,PRICE,DATE_OF_TRANSFER,POSTCODE,OLD_NEW
ADDRESS,,,,
87 THEOBALD ROAD NR1 2NX,37500.0,1995-06-30,NR1 2NX,N
16 AFON RHOS ESTATE LL55 4SE,44500.0,1995-11-22,LL55 4SE,N
69 CRANHILL ROAD BA16 0BZ,48500.0,1995-04-13,BA16 0BZ,N
4 BRIARY LANE SG8 9BZ,53000.0,1995-01-06,SG8 9BZ,N
133 WOOD LANE CH5 3JF,67500.0,1995-12-12,CH5 3JF,N


## Energy Performance of Buildings Data
[***Energy Performance of Buildings Data***](https://epc.opendatacommunities.org/) provides access to *Energy Performance Certificates* and *Display Energy Certificate* data for buildings across England and Wales, based on data released quarterly from 1 October 2008 up to 30 September 2021. The data has to be downloaded manually as it requires authentication in order to access the site.

I will only be requiring the EPC data, which has 90 column headers including:
* Address headers (Address lines 1 to 3)
* Postcode
* Number of habitable rooms
* Total square footage
* Property type

### Sampling data
I will be following a very similar workflow as to processing the *Price Paid data*. Initially, I will use the first certificate file as a sample set and filter out most unnecessary columns. I will then convert the data types of the remaining columns.

In [ ]:
from glob import glob

epcPath = os.path.join(RAW_PATH, "all-domestic-certificates", "*", "certificates.csv")
epcSample = pd.read_csv(glob(epcPath)[0], low_memory=False)

In [ ]:
# hide_input
originalUsage = to_bytes(epcSample.memory_usage(deep=True).sum())
print(originalUsage)
epcSample.head()

207.64MB


,LMK_KEY,ADDRESS1,ADDRESS2,ADDRESS3,POSTCODE,BUILDING_REFERENCE_NUMBER,CURRENT_ENERGY_RATING,POTENTIAL_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,POTENTIAL_ENERGY_EFFICIENCY,...,LOCAL_AUTHORITY_LABEL,CONSTITUENCY_LABEL,POSTTOWN,CONSTRUCTION_AGE_BAND,LODGEMENT_DATETIME,TENURE,FIXED_LIGHTING_OUTLETS_COUNT,LOW_ENERGY_FIXED_LIGHT_COUNT,UPRN,UPRN_SOURCE
0,208945009062019011009444285518421,"15, Elmfield Road",NaN,NaN,GL51 9JJ,2864085568,C,B,71,84,...,Cheltenham,Cheltenham,CHELTENHAM,England and Wales: 1950-1966,2019-01-10 09:44:42,rental (private),NaN,NaN,1.001204e+11,Address Matched
1,1296022739962015031912051214438915,"15, Lime Close",Prestbury,NaN,GL52 3EF,1185324378,E,C,53,79,...,Cheltenham,Tewkesbury,CHELTENHAM,England and Wales: 1967-1975,2015-03-19 12:05:12,owner-occupied,NaN,NaN,1.001204e+11,Address Matched
2,1102892409262017040613095870338213,"44, Naunton Lane",NaN,NaN,GL53 7BH,1795060278,D,B,65,85,...,Cheltenham,Cheltenham,CHELTENHAM,England and Wales: 1900-1929,2017-04-06 13:09:58,owner-occupied,NaN,NaN,1.001204e+11,Address Matched
3,2383300702009033116544446217998,"Flat 8, Hazelhurst","24, Eldorado Road",NaN,GL50 2PT,9774302468,E,D,46,57,...,Cheltenham,Cheltenham,CHELTENHAM,England and Wales: 1900-1929,2009-03-31 16:54:44,rental (private),NaN,NaN,1.000048e+10,Address Matched
4,1656454289042018082020372657982008,"6, Charles Street",NaN,NaN,GL51 9HH,4037189578,C,B,71,87,...,Cheltenham,Cheltenham,CHELTENHAM,England and Wales: before 1900,2018-08-20 20:37:26,owner-occupied,NaN,NaN,1.001204e+11,Address Matched


#### Filtering columns and dropping empty entries
As of now there are far too many unneccessary columns, I will only take the columns I think will potentially be useful as features for predictive models. Additionally, I will be replacing placeholder values such as `"NO DATA!"` to `NaN` values and will subsequently drop them.

In [ ]:
epcFilter = [
    "ADDRESS1",  #*
    "ADDRESS2",  #*
    "POSTCODE",  #*
    "PROPERTY_TYPE",
    "BUILT_FORM",
    "TOTAL_FLOOR_AREA",
    "NUMBER_HABITABLE_ROOMS",
    "EXTENSION_COUNT",
    "GLAZED_TYPE",
    "CONSTRUCTION_AGE_BAND",
    "TENURE"
]
#* WILL LATER BE DROPPED
epcSample = epcSample[epcFilter]

In [ ]:
nanValues = [
    "NULL",
    "INVALID",
    "INVALID!",
    "NODATA!",
    "NO DATA!",
    "N/A",
    "Not applicable",
    "Not recorded",
    "not defined",
    "Blank",
]
epcSample[epcSample.isin(nanValues)] = np.nan
print(epcSample.isna().sum()[lambda x: x > 0])
epcSample = (epcSample
    .dropna(subset=list(filter(lambda x: x != "ADDRESS2", epcFilter)))
    .reset_index(drop=True))

ADDRESS2                  25366
BUILT_FORM                 1141
NUMBER_HABITABLE_ROOMS     4471
EXTENSION_COUNT            4471
GLAZED_TYPE                9938
CONSTRUCTION_AGE_BAND      4460
TENURE                     1432
dtype: int64


#### Converting data types

In [ ]:
float_col = ["TOTAL_FLOOR_AREA", "NUMBER_HABITABLE_ROOMS", "EXTENSION_COUNT"]
cat_col = [
    "PROPERTY_TYPE", "BUILT_FORM", "GLAZED_TYPE", "CONSTRUCTION_AGE_BAND",
    "TENURE"
]
epcSample[float_col] = epcSample[float_col].astype("float")
epcSample[cat_col] = epcSample[cat_col].astype("category")
epcDtype = epcSample.dtypes.to_dict()

#### Formatting addresses
"`ADDRESS1 ADDRESS2 POSTCODE`" all in uppercase, commas removed from text.

In [ ]:
def epcFormatAddr(x):
    return [
        ' '.join(z.strip().replace(",", "").upper() for z in y
                 if not pd.isna(z))
        for y in x[["ADDRESS1", "ADDRESS2", "POSTCODE"]].values
    ]  # List comprehension for performance

In [ ]:
epcSample["ADDRESS"] = epcFormatAddr(epcSample)
epcSample = epcSample.drop(columns=["ADDRESS1", "ADDRESS2", "POSTCODE"])
epcSample.ADDRESS[:5]

0           15 ELMFIELD ROAD GL51 9JJ
1    15 LIME CLOSE PRESTBURY GL52 3EF
2            44 NAUNTON LANE GL53 7BH
3           6 CHARLES STREET GL51 9HH
4          25 BELWORTH COURT GL51 6HQ
Name: ADDRESS, dtype: object

Processed sample set:

In [ ]:
# hide_input
_info = info(epcSample)
print(originalUsage, "=>", to_bytes(_info["memory_usage"].sum()))
_info

207.64MB => 4.50MB


,dtype,memory_usage,null,unique
PROPERTY_TYPE,category,38719,0.0,5.0
BUILT_FORM,category,38823,0.0,6.0
TOTAL_FLOOR_AREA,float64,305808,0.0,7559.0
NUMBER_HABITABLE_ROOMS,float64,305808,0.0,21.0
EXTENSION_COUNT,float64,305808,0.0,5.0
GLAZED_TYPE,category,39180,0.0,8.0
CONSTRUCTION_AGE_BAND,category,39895,0.0,13.0
TENURE,category,39179,0.0,8.0
ADDRESS,object,3390932,0.0,30564.0
Index,NaN,128,NaN,NaN


### Entire dataset
I will be process and export the entire dataset using Dask with a similar pipeline to that of the Price-paid data.

#### Pipeline
1. Import all files as a *glob* using Dask.
2. Drop rows with empty entries (ignoring `ADDRESS2` column)
3. Convert data types
4. Format addresses with `epcFormatAddr` function
7. Drop duplicate addresses and set as index
8. Drop unnecessary columns

In [ ]:
epcFilter = [
    "ADDRESS1", "ADDRESS2", "POSTCODE", "PROPERTY_TYPE", "BUILT_FORM",
    "TOTAL_FLOOR_AREA", "NUMBER_HABITABLE_ROOMS", "EXTENSION_COUNT",
    "GLAZED_TYPE", "CONSTRUCTION_AGE_BAND", "TENURE"
]
nanValues = [
    "NULL",
    "INVALID",
    "INVALID!",
    "NODATA!",
    "NO DATA!",
    "N/A",
    "Not applicable",
    "Not recorded",
    "not defined",
    "Blank",
]
epcDtype = {
    'ADDRESS1': object,
    'ADDRESS2': object,
    'POSTCODE': object,
    'PROPERTY_TYPE': pd.CategoricalDtype(),
    'BUILT_FORM': pd.CategoricalDtype(),
    'TOTAL_FLOOR_AREA': np.float64,
    'NUMBER_HABITABLE_ROOMS': np.float64,
    'EXTENSION_COUNT': np.float64,
    'GLAZED_TYPE': pd.CategoricalDtype(),
    'CONSTRUCTION_AGE_BAND': pd.CategoricalDtype(),
    'TENURE': pd.CategoricalDtype()
}
def epcFormatAddr(x):
    return [
        ' '.join(z.strip().replace(",", "").upper() for z in y
                 if not pd.isna(z))
        for y in x[["ADDRESS1", "ADDRESS2", "POSTCODE"]].values
    ]  # List comprehension for performance

In [ ]:
epcPipeline = (dd
    .read_csv(epcPath, usecols=epcFilter, na_values=nanValues, dtype=epcDtype)
    .dropna(subset=list(filter(lambda x: x != "ADDRESS2", epcFilter)), how="any")
    .map_partitions(lambda x: x.assign(ADDRESS=epcFormatAddr)).drop_duplicates(subset="ADDRESS",keep="last")
    .map_partitions(lambda x: x.set_index("ADDRESS"))
    .drop(columns=["ADDRESS1", "ADDRESS2", "POSTCODE"])
)

In [ ]:
if not os.path.exists(EPC_PATH) and not os.path.exists(DATASET_PATH):
    with ProgressBar():
        epcPipeline.to_parquet(EPC_PATH, compression="snappy")
epc = dd.read_parquet(EPC_PATH)

In [ ]:
# hide_input
print(to_bytes(epc.memory_usage(deep=True).sum().compute()))
print(f"{epc.shape[0].compute():,} rows")
epc.head()

1.64GB
14,148,102 rows


,PROPERTY_TYPE,BUILT_FORM,TOTAL_FLOOR_AREA,GLAZED_TYPE,EXTENSION_COUNT,NUMBER_HABITABLE_ROOMS,CONSTRUCTION_AGE_BAND,TENURE
ADDRESS,,,,,,,,
54 CAISTOR DRIVE TS25 2QG,Bungalow,Semi-Detached,59.0,"double glazing, unknown install date",0.0,4.0,England and Wales: 1967-1975,rental (private)
18 TRENTBROOKE AVENUE TS25 5JN,Bungalow,Semi-Detached,70.8,"double glazing, unknown install date",0.0,4.0,England and Wales: 1950-1966,owner-occupied
3 HARLECH WALK TS26 0TN,House,Mid-Terrace,86.0,double glazing installed before 2002,1.0,5.0,England and Wales: 1950-1966,owner-occupied
31 LAUREL GARDENS TS25 4NZ,Flat,Detached,52.0,double glazing installed during or after 2002,0.0,3.0,England and Wales: 2007 onwards,rental (social)
16 HEADINGLEY COURT SEATON CAREW TS25 2PD,House,Detached,121.0,double glazing installed before 2002,1.0,6.0,England and Wales: 1991-1995,owner-occupied


## Merging datasets

In [ ]:
merge = (dd
    .merge(ppd, epc, how="inner", left_index=True, right_index=True) # intersection of price-paid and certificate data
    .reset_index(drop=True)  # dropping address index as it is no longer necessary
    .map_partitions(lambda x: x.set_index("DATE_OF_TRANSFER").sort_index()) # set date to new index
)

In [ ]:
if not os.path.exists(MERGED_PATH):
    with ProgressBar():
        merge.to_parquet(MERGED_PATH, schema="infer", compression="snappy")
df = dd.read_parquet(MERGED_PATH).compute()  # Computing into memory since it will use ~600mb

In [ ]:
# hide_input
print(to_bytes(df.memory_usage(deep=True).sum()))
print(f"{df.shape[0]:,} rows")
df.head()

614.61MB
5,566,962 rows


,PRICE,POSTCODE,OLD_NEW,PROPERTY_TYPE,BUILT_FORM,TOTAL_FLOOR_AREA,GLAZED_TYPE,EXTENSION_COUNT,NUMBER_HABITABLE_ROOMS,CONSTRUCTION_AGE_BAND,TENURE
DATE_OF_TRANSFER,,,,,,,,,,,
1995-01-01,16000.0,HX1 4NG,N,House,Enclosed Mid-Terrace,99.00,"double glazing, unknown install date",0.0,4.0,England and Wales: before 1900,owner-occupied
1995-01-02,35000.0,NN1 4LL,N,House,Mid-Terrace,85.00,double glazing installed before 2002,1.0,4.0,England and Wales: 1900-1929,owner-occupied
1995-01-03,15000.0,NE34 8TF,N,House,Semi-Detached,74.00,double glazing installed before 2002,0.0,4.0,England and Wales: 1967-1975,rental (private)
1995-01-03,48000.0,HR8 2DB,N,Bungalow,Detached,78.00,double glazing installed before 2002,1.0,3.0,England and Wales: 1950-1966,owner-occupied
1995-01-03,82000.0,TR13 8BP,N,Bungalow,Detached,118.88,double glazing installed during or after 2002,0.0,4.0,England and Wales: 1967-1975,rental (private)


## Downloading additional data for feature engineering
During my dataset analysis, I will require additional data to further transform my dataset. This includes:
* Geographical data from the ***[National Statistics Postcode Lookup (February 2022)](https://geoportal.statistics.gov.uk/datasets/ons::national-statistics-postcode-lookup-february-2022/about)***, e.g longitude, latitude
* HPI (House Price Index) data from ***[National Statistics UK House Price Index (January 2022)](https://www.gov.uk/government/publications/about-the-uk-house-price-index)***

In [ ]:
# collapse
from tqdm import tqdm

class Downloader:
    def __init__(
        self,
        _session,
        pbar_enabled: bool = False,
        pbar_kwargs={},
    ):
        self.session = _session
        self.pbar_enabled = pbar_enabled
        self.pbar_kwargs = pbar_kwargs

    def dlFile(self, file, chunk_size=1024, **reqKwargs) -> None:
        r = self.session.get(
            **reqKwargs, stream=True)  # Sending get request to provided url
        r.raise_for_status() # raise error if request failure
        if self.pbar_enabled:
            pbar = tqdm(desc=file,
                        total=int(r.headers.get("content-length")),
                        unit="iB",
                        unit_scale=True,
                        **self.pbar_kwargs)  # Initialising progress bar
        with open(file, "wb") as f:
            # Iterating chunks as they come from server
            for chunk in r.iter_content(chunk_size=chunk_size):
                f.write(chunk)  # Writing chunk to file
                f.flush()  # Flushing input buffer
                if self.pbar_enabled:
                    pbar.update(len(chunk))  # Updating progress bar by size of chunk
        if self.pbar_enabled: pbar.close()

### Postcode Lookup

In [ ]:
if not os.path.exists(PCD_PATH):
    url = "https://www.arcgis.com/sharing/rest/content/items/1a0444ee3c43452ea16c530966ae8984/data"
    extractPath = PCD_PATH.removesuffix(".csv")
    if not os.path.exists(extractPath):
        zipPath = extractPath + ".zip"
        # downloading zip file from website
        with Session() as sess:
            Downloader(sess, pbar_enabled=True).dlFile(zipPath, url=url)
        # extracting zip to folder
        with zipfile.ZipFile(zipPath, "r") as handle:
            handle.extractall(extractPath)
        os.remove(zipPath)  # removing zip file
    # taking necessary columns and writing into csv file
    (pd
        .read_csv(os.path.join(extractPath, "Data", "NSPL_FEB_2022_UK.csv"), usecols=["pcds", "lat", "long"])
        .rename(columns={"long":"lng"})
        .rename(columns=lambda x: x.upper())
        .to_csv(PCD_PATH, index=False)
    )
    shutil.rmtree(extractPath)  # removing folder
geo = pd.read_csv(PCD_PATH)

In [ ]:
# hide_input
print(to_bytes(geo.memory_usage(deep=True).sum()))
print(f"{geo.shape[0]:,} rows")
geo.head()

214.96MB
2,673,018 rows


,PCDS,LAT,LNG
0,AB1 0AA,57.101474,-2.242851
1,AB1 0AB,57.102554,-2.246308
2,AB1 0AD,57.100556,-2.248342
3,AB1 0AE,57.084444,-2.255708
4,AB1 0AF,57.096656,-2.258102


### House Price Index

In [ ]:
if not os.path.exists(HPI_PATH):
    url = "http://publicdata.landregistry.gov.uk/market-trend-data/house-price-index-data/Indices-2022-02.csv"
    # Parsing file and writing to csv file
    (pd.read_csv(url, usecols=["Date", "Region_Name", "Index"],parse_dates=["Date"])
        .query("Region_Name == 'England and Wales'", engine="python")
        .set_index("Date").sort_index()
        .Index.to_csv(HPI_PATH)
    )
hpi = pd.read_csv(HPI_PATH)

In [ ]:
# hide_input
print(to_bytes(hpi.memory_usage(deep=True).sum()))
print(f"{hpi.shape[0]:,} rows")
hpi.head()

24.58KB
326 rows


,Date,Index
0,1995-01-01,26.440075
1,1995-02-01,26.369132
2,1995-03-01,26.401129
3,1995-04-01,26.604794
4,1995-05-01,26.645437


All the necessary data has now been collected locally, however it is still unusable and thus useless; I will now move onto exploring and analysing the data, performing transformations along the way, in order to make the data better fit and explain the price data.